In [1]:
import json
import os

In [2]:
data = []
with open('dictionary.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        data.append(l)

In [3]:
len(data)

54712

In [4]:
data[0]

'[ke.pa.ya.han]  |  کڤايهنDefinisi : 1. perihal payah, kesukaran, ke\xadsusahan: Wahab mendengar cerita-cerita tentang ~ penduduk pekan Tembeling akibat sekatan yg diadakan oleh kerajaan; 2. men\xadderita payah (kesulitan, kekurangan): sedangkan hendak mendapat sesuap nasi ~, inikan pula hendak menziarahi tempat-tempat yg bersejarah; 3. kelelahan, keletihan: beberapa lamanya ia berdiam diri saja, terengah-engah spt orang ~.\xa0(Kamus Dewan Edisi Keempat)\n[ke.pa.ya.han]  |  کڤايهنDefinisi : hal atau keadaan payah; kesukaran; kesusahan: ~ penduduk kampung tsb berhubung dgn kawasan bandar telah mendapat perhatian drpd pihak kerajaan.\xa0(Kamus Pelajar Edisi Kedua)\nberpayah-payah\nmemayahkan\nmemperpayah\nkepayahan'

In [5]:
folder = 'dictionary-qa-llama3'

In [6]:
# !rm -rf generate-wikipedia-qa-llama3
!mkdir {folder}

mkdir: cannot create directory ‘dictionary-qa-llama3’: File exists


In [7]:
results = []
for i in range(len(data)):
    
    s = """
```
{{replace_me}}
```

Generate list of QA choice in malay, return in JSON [{'question', 'A', 'B', 'C', 'D', 'answer'}]
    """.strip()
    s = s.replace('{{replace_me}}', data[i])
    results.append(s)
        
len(results)

54712

In [8]:
from openai import OpenAI
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

def answer(q, i):
    filename = os.path.join(folder, f'{i}.json')
    if os.path.exists(filename):
        return
    
    results = []
    openai = OpenAI(
        base_url='https://llama-3.us.mesolitica.com/v1',
        api_key='empty',
    )
    for _ in range(1):
        
        while True:
            try:
                
                json_data = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': q,
                        },
                    ],
                    'model': 'model',
                    'stop': [
                        '<|eot_id|>',
                    ],
                    'temperature': 0.9,
                    'max_tokens': 2048,
                }
                response = requests.post(
                    'https://llama-3.us.mesolitica.com/v1/chat/completions', 
                    headers=headers, json=json_data, timeout = 60 * 10)
                
                r = response.json()['choices'][0]['message']['content']
                results.append(r)
                break
            except Exception as e:
                # print(e)
                pass
                
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

In [9]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [10]:
urls = [(q, no) for no, q in enumerate(results)]

In [11]:
answer(*urls[0])

In [12]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

  0%|                                                                                       | 0/54712 [00:00<?, ?it/s]

In [7]:
data = []
with open('dictionary.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        data.append({
            'text': l
        })

In [8]:
data[0]

{'text': '[ke.pa.ya.han]  |  کڤايهنDefinisi : 1. perihal payah, kesukaran, ke\xadsusahan: Wahab mendengar cerita-cerita tentang ~ penduduk pekan Tembeling akibat sekatan yg diadakan oleh kerajaan; 2. men\xadderita payah (kesulitan, kekurangan): sedangkan hendak mendapat sesuap nasi ~, inikan pula hendak menziarahi tempat-tempat yg bersejarah; 3. kelelahan, keletihan: beberapa lamanya ia berdiam diri saja, terengah-engah spt orang ~.\xa0(Kamus Dewan Edisi Keempat)\n[ke.pa.ya.han]  |  کڤايهنDefinisi : hal atau keadaan payah; kesukaran; kesusahan: ~ penduduk kampung tsb berhubung dgn kawasan bandar telah mendapat perhatian drpd pihak kerajaan.\xa0(Kamus Pelajar Edisi Kedua)\nberpayah-payah\nmemayahkan\nmemperpayah\nkepayahan'}

In [10]:
suppose = {'question', 'A', 'B', 'C', 'D', 'answer'}

In [12]:
from tqdm import tqdm

with open('generated-dictionary-qa.jsonl', 'w') as fopen_j:
    for i in tqdm(range(len(data))):
        f = f'{folder}/{i}.json'
        if not os.path.exists(f):
            continue
            
        with open(f) as fopen:
            d = json.load(fopen)
        
        questions = []
        unique_questions = set()
        for d_ in d:
            try:
                selected = []
                for d__ in d_.split('```'):
                    try:
                        try:
                            selected.append(json.loads(d__))
                        except:
                            selected.append(eval(d__))
                    except:
                        pass
                if not len(selected):
                    continue
                qa = selected[0]
                for q in qa:
                    if len(set(q.keys()) | suppose) != len(suppose):
                        continue
                    if q['question'].lower() not in unique_questions:
                        if not ('A' in q and isinstance(q['A'], str)):
                            continue
                        if not ('B' in q and isinstance(q['B'], str)):
                            continue
                        if not ('C' in q and isinstance(q['C'], str)):
                            continue
                        if not ('D' in q and isinstance(q['D'], str)):
                            continue
                        if not ('answer' in q and isinstance(q['answer'], str)):
                            continue
                            
                        for k in suppose:
                            q[k] = q[k].encode("utf8", errors="surrogatepass").decode("utf8")

                        questions.append(q)
                        unique_questions.add(q['question'].lower())
            except:
                pass
            
        if not len(questions):
            continue
        
        data[i]['text'] = data[i]['text'].encode("utf8", errors="surrogatepass").decode("utf8")
        data[i]['questions'] = questions
        fopen_j.write(f'{json.dumps(data[i])}\n')

100%|█████████████████████████████████████████████████████████████████████████| 54712/54712 [00:07<00:00, 7675.79it/s]


In [13]:
!wc -l generated-dictionary-qa.jsonl

48674 generated-dictionary-qa.jsonl


In [14]:
!ls -lh generated-dictionary-qa.jsonl

-rw-rw-r-- 1 husein husein 54M Mei  30 14:24 generated-dictionary-qa.jsonl


In [15]:
!head -n 1 generated-dictionary-qa.jsonl

{"text": "[ke.pa.ya.han]  |  \u06a9\u06a4\u0627\u064a\u0647\u0646Definisi : 1. perihal payah, kesukaran, ke\u00adsusahan: Wahab mendengar cerita-cerita tentang ~ penduduk pekan Tembeling akibat sekatan yg diadakan oleh kerajaan; 2. men\u00adderita payah (kesulitan, kekurangan): sedangkan hendak mendapat sesuap nasi ~, inikan pula hendak menziarahi tempat-tempat yg bersejarah; 3. kelelahan, keletihan: beberapa lamanya ia berdiam diri saja, terengah-engah spt orang ~.\u00a0(Kamus Dewan Edisi Keempat)\n[ke.pa.ya.han]  |  \u06a9\u06a4\u0627\u064a\u0647\u0646Definisi : hal atau keadaan payah; kesukaran; kesusahan: ~ penduduk kampung tsb berhubung dgn kawasan bandar telah mendapat perhatian drpd pihak kerajaan.\u00a0(Kamus Pelajar Edisi Kedua)\nberpayah-payah\nmemayahkan\nmemperpayah\nkepayahan", "questions": [{"question": "Apa maksud kepayahan dalam konteks cerita?", "A": "Kesusahan dan kesukaran", "B": "Kelelahan dan keletihan", "C": "Kesenangan dan kegembiraan", "D": "Kebebasan dan kebaha

In [16]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='generated-dictionary-qa.jsonl',
    path_in_repo='generated-dictionary-qa.jsonl',
    repo_id='mesolitica/llama-3-70b-qa',
    repo_type='dataset',
)

generated-dictionary-qa.jsonl:   0%|          | 0.00/56.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/llama-3-70b-qa/commit/37d0e4c31523d253dd26659506d52702898a4a78', commit_message='Upload generated-dictionary-qa.jsonl with huggingface_hub', commit_description='', oid='37d0e4c31523d253dd26659506d52702898a4a78', pr_url=None, pr_revision=None, pr_num=None)